# IMPORT LIBRIARIES

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Load carbon emission data

In [ ]:
carb=pd.read_csv("/kaggle/input/carbon-emissions-and-population-from-1961-to-2015/carbon.csv")
carbon_data=carb[["Year","total_population","Avg_Carbon_emmsion_per capita","avg_temp_change"]]
carbon_data["total_population"]=carbon_data["total_population"].apply(pd.np.log)
carbon_data.head()

## Load weather Data

In [ ]:
weather_data=pd.read_csv("/kaggle/input/rainfall-in-india/rainfall in india 1901-2015.csv")

## filter Year greater than 1960
weather_data=weather_data[weather_data["YEAR"]>1960]

## filter data based on Subdivision
weather_data=weather_data[weather_data["SUBDIVISION"]=='COASTAL KARNATAKA']

weather_data=weather_data.fillna(value=0)
weather_data.index=weather_data["YEAR"]
weather_data.head()

### unstack data columns "Months" into roles

In [ ]:
_data=weather_data[
    ['JAN',
     'FEB',
     'MAR',
     'APR',
     'MAY',
     'JUN',
     'JUL',
     'AUG',
     'SEP',
     'OCT',
     'NOV',
     'DEC']]
_data.columns=['01-JAN',
 '02-FEB',
 '03-MAR',
 '04-APR',
 '05-MAY',
 '06-JUN',
 '07-JUL',
 '08-AUG',
 '09-SEP',
 '10-OCT',
 '11-NOV',
 "12-DEC"]
_data=_data.unstack().reset_index(name="value")

### Merge the carbon and weather dataset

In [ ]:
data=pd.merge(_data.sort_values(by=['YEAR','level_0']),carbon_data,left_on="YEAR",right_on="Year")[['level_0',
 'Year',"value",
 'total_population',
 'Avg_Carbon_emmsion_per capita',
 'avg_temp_change']]

# Data exploration

In [ ]:
data.describe()

## data split

In [ ]:
test_data=data[501:]
train_data=data[:501]

In [ ]:
header=list(train_data.columns)
target="value"
header.remove(target)

## Hot Encode data

In [ ]:
import category_encoders as ce
encoder=ce.OneHotEncoder()
encoder.fit(train_data[header],train_data[target])
transformed_train=encoder.transform(train_data[header])

In [ ]:
data.hist(figsize = (12,10))
plt.show()

# Neutral Network Model

In [ ]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(32, kernel_initializer='normal',input_dim = transformed_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(16, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(8, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(4, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

## Save models

In [ ]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

## Train Model

In [ ]:
NN_model.fit(transformed_train, train_data[target], epochs=800, batch_size=12, validation_split = 0.2, callbacks=callbacks_list)

## Download weights

In [ ]:
from IPython.display import FileLinks
FileLinks(".")

# Predict  this month's  rainfall

In [ ]:
NN_model.predict(np.array([[0,0	,0,	0,	0,	0,	1,	0,	0,	0,	0,	0,	2021,	24.910377,	4.621587,	0.926928]]))[0][0]

# Predict  Next month's  rainfall

In [ ]:
NN_model.predict(np.array([[0,0 ,0, 0,  0,  0,  0,  1,  0,  0,  0,  0,  2021,   24.910377,  4.621587,   0.926928]]))[0][0]